In [ ]:
###Imports

from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import os
import random
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

#Imports for models
import tensorflow as tf 
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Activation, Input, Conv2DTranspose, Lambda
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from skimage.io import imread, imshow
from skimage.transform import resize

#seed = 42
#np.random.seed = seed


In [ ]:
def initDataset(folder, classes=None, mode='train'):    
    ##Initialize COCO api for instance annotations
    annFile = '{}/annotations.json'.format(folder)
    coco = COCO(annFile)
    
    images = []

    ##Judging for train/val datasets
    if mode == 'train':
        imgIds = coco.getImgIds(range(0, 1200))
        images = coco.loadImgs(imgIds)

    elif mode == 'val':
        imgIds = coco.getImgIds(range(1200, 1500))
        images = coco.loadImgs(imgIds)
                
    random.shuffle(images)
    dataset_size = len(images)
    
    return images, dataset_size, coco

In [ ]:
annFile = '/content/drive/My Drive/datasets/TACO_Litter/TACO/data/annotations.json'
coco_cats = COCO(annFile)
catIds = coco_cats.getCatIds()
cats = coco_cats.loadCats(catIds)

nms = [cat['name'] for cat in cats]
print(len(nms), "COCO Categories: \n {} \n".format(''.join(nms)))
print(nms)


loading annotations into memory...
Done (t=2.35s)
creating index...
index created!
60 COCO Categories: 
 Aluminium foilBatteryAluminium blister packCarded blister packOther plastic bottleClear plastic bottleGlass bottlePlastic bottle capMetal bottle capBroken glassFood CanAerosolDrink canToilet tubeOther cartonEgg cartonDrink cartonCorrugated cartonMeal cartonPizza boxPaper cupDisposable plastic cupFoam cupGlass cupOther plastic cupFood wasteGlass jarPlastic lidMetal lidOther plasticMagazine paperTissuesWrapping paperNormal paperPaper bagPlastified paper bagPlastic filmSix pack ringsGarbage bagOther plastic wrapperSingle-use carrier bagPolypropylene bagCrisp packetSpread tubTupperwareDisposable food containerFoam food containerOther plastic containerPlastic gloovesPlastic utensilsPop tabRope & stringsScrap metalShoeSqueezable tubePlastic strawPaper strawStyrofoam pieceUnlabeled litterCigarette 

['Aluminium foil', 'Battery', 'Aluminium blister pack', 'Carded blister pack', 'Other plast

In [ ]:
def getClassName(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return None

def getImage(imageObj, img_folder, input_image_size):
    ##Read and normalize an image
    train_img = io.imread(img_folder + '/' + imageObj['file_name'])/255.0
    ##Resize
    train_img = cv2.resize(train_img, input_image_size)
    if (len(train_img.shape)==3 and train_img.shape[2]==3): ##If it is a RGB 3 channel image
        return train_img
    else: ##To handle a black and white image, increase dimensions to 3
        stacked_img = np.stack((train_img,)*3, axis=-1)
        return stacked_img
    
def getNormalMask(imageObj, classes, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    cats = coco.loadCats(catIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        className = getClassName(anns[a]['category_id'], cats)
        pixel_value = classes.index(className)+1
        new_mask = cv2.resize(coco.annToMask(anns[a])*pixel_value, input_image_size)
        train_mask = np.maximum(new_mask, train_mask)

    ##Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask  
    
def getBinaryMask(imageObj, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        new_mask = cv2.resize(coco.annToMask(anns[a]), input_image_size)
        
        ##Threshold because resizing may cause extraneous values
        new_mask[new_mask >= 0.5] = 1
        new_mask[new_mask < 0.5] = 0

        train_mask = np.maximum(new_mask, train_mask)

    ##Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask


def dataGeneratorCoco(images, classes, coco, folder, 
                      input_image_size=(224,224), batch_size=4, mode='train', mask_type='binary'):
    
    img_folder = folder + '/'
    dataset_size = len(images)
    catIds = coco.getCatIds()
    
    c = 0
    while(True):
        img = np.zeros((batch_size, input_image_size[0], input_image_size[1], 3)).astype('float')
        mask = np.zeros((batch_size, input_image_size[0], input_image_size[1], 1)).astype('float')

        for i in range(c, c+batch_size): ##Initially from 0 to batch_size, when c = 0
            imageObj = images[i]
            
            ###Retrieve Image
            train_img = getImage(imageObj, img_folder, input_image_size)
            
            ###Create Mask
            if mask_type=="binary":
                train_mask = getBinaryMask(imageObj, coco, catIds, input_image_size)
            
            elif mask_type=="normal":
                train_mask = getNormalMask(imageObj, classes, coco, catIds, input_image_size)                
            
            ##Add to respective batch sized arrays
            img[i-c] = train_img
            mask[i-c] = train_mask
            
        c+=batch_size
        if(c + batch_size >= dataset_size):
            c=0
            random.shuffle(images)
        yield img, mask

In [ ]:
def augmentationsGenerator(gen, augGeneratorArgs, seed=None):
    ##Initialize the image data generator with args provided
    image_gen = ImageDataGenerator(**augGeneratorArgs)
    
    ##Remove the brightness argument for the mask. Spatial arguments similar to image.
    augGeneratorArgs_mask = augGeneratorArgs.copy()
    _ = augGeneratorArgs_mask.pop('brightness_range', None)

    ##Initialize the mask data generator with modified args
    mask_gen = ImageDataGenerator(**augGeneratorArgs_mask)
    
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    
    for img, mask in gen:
        seed = np.random.choice(range(9999))
        ##Keeping the seeds syncronized otherwise the augmentation of the images will end up different from the augmentation of the masks
        g_x = image_gen.flow(255*img, 
                             batch_size = img.shape[0], 
                             seed = seed, 
                             shuffle=True)
        g_y = mask_gen.flow(mask, 
                             batch_size = mask.shape[0], 
                             seed = seed, 
                             shuffle=True)
        
        img_aug = next(g_x)/255.0
        
        mask_aug = next(g_y)
                   

        yield img_aug, mask_aug

In [ ]:
augGeneratorArgs = dict(featurewise_center = False, 
                        samplewise_center = False,
                        rotation_range = 5, 
                        width_shift_range = 0.01, 
                        height_shift_range = 0.01, 
                        brightness_range = (0.8,1.2),
                        shear_range = 0.01,
                        zoom_range = [1, 1.25],  
                        horizontal_flip = True, 
                        vertical_flip = False,
                        fill_mode = 'reflect',
                        data_format = 'channels_last')


In [ ]:
def visualizeGenerator(gen):
    img, mask = next(gen)
    
    fig = plt.figure(figsize=(20, 10))
    outerGrid = gridspec.GridSpec(1, 2, wspace=0.1, hspace=0.1)
    
    for i in range(2):
        innerGrid = gridspec.GridSpecFromSubplotSpec(2, 2,
                        subplot_spec=outerGrid[i], wspace=0.05, hspace=0.05)

        for j in range(4):
            ax = plt.Subplot(fig, innerGrid[j])
            if(i==1):
                ax.imshow(img[j])
            else:
                ax.imshow(mask[j][:,:,0])
                
            ax.axis('off')
            fig.add_subplot(ax)        
    plt.show()

In [ ]:
visualizeGenerator(train_gen)

In [ ]:
###Model time!

#Defining variables

annFile = '/content/drive/My Drive/datasets/TACO_Litter/TACO/data/annotations.json'
coco_cats = COCO(annFile)
catIds = coco_cats.getCatIds()
cats = coco_cats.loadCats(catIds)

_classes = nms
train = 'train'
val = 'val'
input_image_size = (224, 224)
batch_size = 8
binary = 'binary'
normal = 'normal'
dataDir = '/content/drive/My Drive/datasets/TACO_Litter/TACO/data'


##Creating train dataset
train_set, train_size, coco_train = initDataset(dataDir, _classes, train)

##Creating val dataset
val_set, val_size, coco_val = initDataset(dataDir, _classes, val)

##Train generator
#val_gen = dataGeneratorCoco(images, classes, coco, dataDir, input_image_size, batch_size, mode, mask_type)
train_gen = dataGeneratorCoco(train_set, _classes, coco_train, dataDir, input_image_size, batch_size, train, normal)

##Val generator
val_gen = dataGeneratorCoco(val_set, _classes, coco_val, dataDir, input_image_size, batch_size, val, normal)

##Augmenting train generator
train_gen_aug = augmentationsGenerator(train_gen, augGeneratorArgs)

##Augmenting val generator
val_gen_aug = augmentationsGenerator(val_gen, augGeneratorArgs)

print(train_size, val_size)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
1200 300


In [ ]:
##Resetting and defining logs
!rm -rf '/content/drive/My Drive/THESIS/taco_logs3'
!mkdir '/content/drive/My Drive/THESIS/taco_logs3'
logs = '/content/drive/My Drive/THESIS/taco_logs3'

In [ ]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_CHANNELS = 3
epochs = 25
validation_steps = val_size
steps_per_epoch = train_size
x = 64

##Creating the model

initializer = "he_normal"

###Building U-Net Model

##Input Layer
inputs = Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))

##Converting inputs to float
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

##Contraction
c1 = tf.keras.layers.Conv2D(x, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(x, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c1)
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

c2 = tf.keras.layers.Conv2D(x*2, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(x*2, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(x*4, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(x*4, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(x*8, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(x*8, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c4)
p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)

c5 = tf.keras.layers.Conv2D(x*16, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(x*16, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c5)

##Expansion
u6 = tf.keras.layers.Conv2DTranspose(x*8, (2,2), strides=(2,2), padding="same")(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(x*8, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(x*8, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c6)

u7 = tf.keras.layers.Conv2DTranspose(x*4, (2,2), strides=(2,2), padding="same")(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(x*4, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(x*4, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c7)

u8 = tf.keras.layers.Conv2DTranspose(x*2, (2,2), strides=(2,2), padding="same")(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(x*2, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(x*2, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c8)

u9 = tf.keras.layers.Conv2DTranspose(x, (2,2), strides=(2,2), padding="same")(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(x, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(x, (3,3), activation="relu", kernel_initializer=initializer, padding="same")(c9)

##Output Layer
outputs = tf.keras.layers.Conv2D(61, (1,1), activation="softmax")(c9)

##Defining Model
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

##Compiling Model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

##Calling Model Summary
model.summary()

##Defining callbacks
callbacks = [
             tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/THESIS/taco_2-2_final_retry_64.h5', verbose=1, save_best_only=True),
             tf.keras.callbacks.EarlyStopping(patience=6, monitor="val_loss"),
             tf.keras.callbacks.TensorBoard(log_dir=logs),
             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience = 2, min_lr = 0.001)]

##Training the modelEpoch 1/25
results = model.fit(x = train_gen_aug, 
                    validation_data = val_gen_aug, 
                    steps_per_epoch = steps_per_epoch, 
                    validation_steps = validation_steps, 
                    epochs = epochs, 
                    callbacks=callbacks,
                    verbose = True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 64) 1792        lambda[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 224, 224, 64) 0           conv2d[0][0]                     
______________________________________________________________________________________________

In [ ]:
###Loading partially trained model and continuing training
taco = tf.keras.models.load_model('/content/drive/My Drive/THESIS/taco_2-2_final_retry.h5')
history = taco.fit(x = train_gen_aug, 
                    validation_data = val_gen_aug, 
                    steps_per_epoch = steps_per_epoch, 
                    validation_steps = validation_steps, 
                    epochs = epochs, 
                    callbacks=callbacks,
                    verbose = True)
